In [2]:
from decouple import config

In [3]:
assert config('ALPHA_VANTAGE_API_KEY', default=None, cast=str) is not None


In [4]:
ALPHA_VANTAGE_API_KEY = config('ALPHA_VANTAGE_API_KEY', default=None, cast=str)

In [5]:
import requests

api_key = ALPHA_VANTAGE_API_KEY

params = {
    'api_key': ALPHA_VANTAGE_API_KEY,
    'ticker': 'AAPL',
    'function': 'TIME_SERIES_INTRADAY'
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

data


{'Meta Data': {'1. Information': 'Intraday (1min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2025-08-13 19:59:00',
  '4. Interval': '1min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (1min)': {'2025-08-13 19:59:00': {'1. open': '233.2000',
   '2. high': '233.3000',
   '3. low': '233.1000',
   '4. close': '233.3000',
   '5. volume': '1507'},
  '2025-08-13 19:58:00': {'1. open': '233.2400',
   '2. high': '233.2500',
   '3. low': '233.2000',
   '4. close': '233.2000',
   '5. volume': '1874'},
  '2025-08-13 19:57:00': {'1. open': '233.2100',
   '2. high': '233.2400',
   '3. low': '233.2000',
   '4. close': '233.2400',
   '5. volume': '110'},
  '2025-08-13 19:56:00': {'1. open': '233.1300',
   '2. high': '233.2500',
   '3. low': '233.1300',
   '4. close': '233.2500',
   '5. volume': '1359'},
  '2025-08-13 19:55:00': {'1. open': '233.1600',
   '2. high': '233.1800',
   '3. low': '233.1300',
   '4. close': '233.

In [6]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == 'meta data'][0]
dataset_key

'Time Series (1min)'

In [7]:
results = data[dataset_key]
for timestamp_str in results.keys():
    print(timestamp_str, results.get(timestamp_str))

2025-08-13 19:59:00 {'1. open': '233.2000', '2. high': '233.3000', '3. low': '233.1000', '4. close': '233.3000', '5. volume': '1507'}
2025-08-13 19:58:00 {'1. open': '233.2400', '2. high': '233.2500', '3. low': '233.2000', '4. close': '233.2000', '5. volume': '1874'}
2025-08-13 19:57:00 {'1. open': '233.2100', '2. high': '233.2400', '3. low': '233.2000', '4. close': '233.2400', '5. volume': '110'}
2025-08-13 19:56:00 {'1. open': '233.1300', '2. high': '233.2500', '3. low': '233.1300', '4. close': '233.2500', '5. volume': '1359'}
2025-08-13 19:55:00 {'1. open': '233.1600', '2. high': '233.1800', '3. low': '233.1300', '4. close': '233.1317', '5. volume': '404'}
2025-08-13 19:54:00 {'1. open': '233.1988', '2. high': '233.1988', '3. low': '233.1600', '4. close': '233.1600', '5. volume': '183'}
2025-08-13 19:53:00 {'1. open': '233.1000', '2. high': '233.2000', '3. low': '233.1000', '4. close': '233.2000', '5. volume': '825'}
2025-08-13 19:52:00 {'1. open': '233.1300', '2. high': '233.1300',

In [17]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-08-13 19:59:00'

In [27]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '233.2000',
 '2. high': '233.3000',
 '3. low': '233.1000',
 '4. close': '233.3000',
 '5. volume': '1507'}

In [30]:
import pytz
from datetime import datetime
from decimal import Decimal
def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern = pytz.timezone('US/Eastern')
    utc = pytz.utc
    timestamp = eastern.localize(
        datetime.strptime(timestamp_str, timestamp_format)
    ).astimezone(utc)
    # unix_timestampt = result.get('t') / 1000.0
    # utc_timestampt = datetime.fromtimestamp(unix_timestampt, tz=pytz.timezone('UTC'))
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': Decimal(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp
    }

In [32]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('233.2000'),
 'close_price': Decimal('233.3000'),
 'high_price': Decimal('233.3000'),
 'low_price': Decimal('233.1000'),
 'number_of_trades': None,
 'volume': Decimal('1507'),
 'volume_weighted_average': None,
 'time': datetime.datetime(2025, 8, 13, 23, 59, tzinfo=<UTC>)}

In [33]:
for timestamp_str in results.keys():
    new_date = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_date)

{'open_price': Decimal('233.2000'), 'close_price': Decimal('233.3000'), 'high_price': Decimal('233.3000'), 'low_price': Decimal('233.1000'), 'number_of_trades': None, 'volume': Decimal('1507'), 'volume_weighted_average': None, 'time': datetime.datetime(2025, 8, 13, 23, 59, tzinfo=<UTC>)}
{'open_price': Decimal('233.2400'), 'close_price': Decimal('233.2000'), 'high_price': Decimal('233.2500'), 'low_price': Decimal('233.2000'), 'number_of_trades': None, 'volume': Decimal('1874'), 'volume_weighted_average': None, 'time': datetime.datetime(2025, 8, 13, 23, 58, tzinfo=<UTC>)}
{'open_price': Decimal('233.2100'), 'close_price': Decimal('233.2400'), 'high_price': Decimal('233.2400'), 'low_price': Decimal('233.2000'), 'number_of_trades': None, 'volume': Decimal('110'), 'volume_weighted_average': None, 'time': datetime.datetime(2025, 8, 13, 23, 57, tzinfo=<UTC>)}
{'open_price': Decimal('233.1300'), 'close_price': Decimal('233.2500'), 'high_price': Decimal('233.2500'), 'low_price': Decimal('233.1